In [1]:
import os,sys
sys.path.append("../../")
from analysis_common import *
from root_common import *
from analysis_helpers import *
%jsroot on

#Definitions
lep_accept_pt = 0.4 #GeV
lep_accept_angle = radians(40)
hadron_accept_ke = 0.060 #GeV
hadron_accept_angle = radians(40)

dir_files_G18_02a = "/Data/ldmx_eTi_4GeV_G18_02a_00"
dir_files_GEM21_11b = "/Data/ldmx_eTi_4GeV_GEM21_11b_00"
gst_files_G18_02a = glob.glob(f"{dir_files_G18_02a}/*gst.root")
gst_files_GEM21_11b = glob.glob(f"{dir_files_GEM21_11b}/*gst.root")

gst_chain_GEM21_11b = create_gst_chain(gst_files_GEM21_11b,verbose=True)
gst_chain_G18_02a = create_gst_chain(gst_files_G18_02a,verbose=True)

events = 100000
df_gst_GEM21_11b_all = ROOT.RDataFrame(gst_chain_GEM21_11b)
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Range(events)

df_gst_G18_02a_all = ROOT.RDataFrame(gst_chain_GEM21_11b)
df_gst_G18_02a_all = df_gst_G18_02a_all.Range(events)

Welcome to JupyROOT 6.24/06
Created gst chain from 10 files with 1000000 total events.
Created gst chain from 10 files with 1000000 total events.


In [2]:
#define a function that makes new lepton variables
def define_df_gst_lep_vars(df_gst):
    df_gst = df_gst.Define("ptl","sqrt(pxl*pxl+pyl*pyl)") #pt lepton
    df_gst = df_gst.Define("thetazl","atan2(ptl,pzl)") #theta_z of lepton
    df_gst = df_gst.Define("energy_transfer","Ev-El")
    return df_gst
#apply definitions for new lepton variables
df_gst_GEM21_11b_all = define_df_gst_lep_vars(df_gst_GEM21_11b_all)

#define a function to make a bunch of hadron related variables
#sfx is the suffix on the vars (initial, final)
def define_df_gst_hadron_vars(df_gst,sfx=["i","f"]):
    
    for s in sfx:
        df_gst = df_gst.Define(f"thetaxz{s}",f"atan2(px{s},pz{s})")
        df_gst = df_gst.Define(f"thetayz{s}",f"atan2(py{s},pz{s})")
        df_gst = df_gst.Define(f"pt{s}",f"sqrt(px{s}*px{s}+py{s}*py{s})")
        
        #note, only for 'i' is total momentum not already in the gst tree
        if s=="i":
            df_gst = df_gst.Define(f"p{s}",f"sqrt(px{s}*px{s}+py{s}+py{s}+pz{s}*pz{s})")
            
        df_gst = df_gst.Define(f"thetaz{s}",f"atan2(pt{s},pz{s})")
        df_gst = df_gst.Define(f"mass{s}",f"sqrt(E{s}*E{s}-p{s}*p{s})")
        df_gst = df_gst.Define(f"ke{s}",f"E{s}-mass{s}")
        
        #for the hadrons, get the indices sorted by KE
        df_gst = df_gst.Define(f"idx_ke{s}",f"Reverse(Argsort(ke{s}))")
                
    return df_gst
df_gst_GEM21_11b_all = define_df_gst_hadron_vars(df_gst_GEM21_11b_all)
c = ROOT.TCanvas("c")

In [3]:
df_gst_GEM21_11b_accept = df_gst_GEM21_11b_all.Filter(f"ptl>{lep_accept_pt} && abs(thetazl)<{lep_accept_angle}")
h_lep_pt_QEL = df_gst_GEM21_11b_accept.Filter("qel==1").Histo1D(("Lepton Angle", "Angles in Acceptance Range", 100, 0, 3.0), "ptl")
h_lep_pt_QEL.SetLineColor(ROOT.kBlue)
h_lep_pt_QEL.Draw("same")

h_lep_pt_MEC = df_gst_GEM21_11b_accept.Filter("mec==1").Histo1D(("Lepton Angle", "", 100, 0, 3.0), "ptl")
h_lep_pt_MEC.SetLineColor(ROOT.kRed)
h_lep_pt_MEC.Draw("same")

h_lep_pt_RES = df_gst_GEM21_11b_accept.Filter("res==1").Histo1D(("Lepton Angle", "", 100, 0, 3.0), "ptl")
h_lep_pt_RES.SetLineColor(ROOT.kGreen)
h_lep_pt_RES.Draw("same")

h_lep_pt_DIS = df_gst_GEM21_11b_accept.Filter("dis==1").Histo1D(("Lepton Angle", "", 100, 0, 3.0), "ptl")
h_lep_pt_DIS.SetLineColor(ROOT.kViolet)
h_lep_pt_DIS.Draw("same")

leg = ROOT.TLegend(0.8, 0.8, 0.8, 0.8)

leg.AddEntry("h_lep_pt_QEL", "QE", "l")
leg.AddEntry("h_lep_pt_MEC", "MEC", "l")
leg.AddEntry("h_lep_pt_RES", "RES", "l")
leg.AddEntry("h_lep_pt_DIS", "DIS", "l")
leg.Draw()
c.Draw()

Warning in <TLegend::Paint>: Legend too large to be automatically placed; a default position is used


In [4]:
d = ROOT.TCanvas("d")
df_gst_G18_02a_accept = df_gst_G18_02a_all.Filter(f"ptl>{lep_accept_pt} && abs(thetazl)<{lep_accept_angle}")
h18_lep_pt_QEL = df_gst_G18_02a_accept.Filter("qel==1").Histo1D(("Lepton Angle", "Angles in Acceptance Range", 100, 0, 3.0), "ptl")
h18_lep_pt_QEL.SetLineColor(ROOT.kBlue)
h18_lep_pt_QEL.Draw("same")

h18_lep_pt_MEC = df_gst_G18_02a_accept.Filter("mec==1").Histo1D(("Lepton Angle", "", 100, 0, 3.0), "ptl")
h18_lep_pt_MEC.SetLineColor(ROOT.kRed)
h18_lep_pt_MEC.Draw("same")

h18_lep_pt_RES = df_gst_G18_02a_accept.Filter("res==1").Histo1D(("Lepton Angle", "", 100, 0, 3.0), "ptl")
h18_lep_pt_RES.SetLineColor(ROOT.kGreen)
h18_lep_pt_RES.Draw("same")

h18_lep_pt_DIS = df_gst_G18_02a_accept.Filter("dis==1").Histo1D(("Lepton Angle", "", 100, 0, 3.0), "ptl")
h18_lep_pt_DIS.SetLineColor(ROOT.kViolet)
h18_lep_pt_DIS.Draw("same")

legg = ROOT.TLegend(0.8, 0.8, 0.8, 0.8)

legg.AddEntry("h_lep_pt_QEL", "QE", "l")
legg.AddEntry("h_lep_pt_MEC", "MEC", "l")
legg.AddEntry("h_lep_pt_RES", "RES", "l")
legg.AddEntry("h_lep_pt_DIS", "DIS", "l")
legg.Draw()
d.Draw()

runtime_error: Template method resolution failed:
  ROOT::RDF::RInterface<ROOT::Detail::RDF::RJittedFilter,void> ROOT::RDF::RInterface<ROOT::Detail::RDF::RRange<ROOT::Detail::RDF::RLoopManager>,void>::Filter(experimental::basic_string_view<char,char_traits<char> > expression, experimental::basic_string_view<char,char_traits<char> > name = "") =>
    runtime_error: 
RDataFrame: An error occurred during just-in-time compilation. The lines above might indicate the cause of the crash
 All RDF objects that have not run an event loop yet should be considered in an invalid state.

  ROOT::RDF::RInterface<ROOT::Detail::RDF::RJittedFilter,void> ROOT::RDF::RInterface<ROOT::Detail::RDF::RRange<ROOT::Detail::RDF::RLoopManager>,void>::Filter(experimental::basic_string_view<char,char_traits<char> > expression, experimental::basic_string_view<char,char_traits<char> > name = "") =>
    runtime_error: 
RDataFrame: An error occurred during just-in-time compilation. The lines above might indicate the cau

input_line_125:2:29: error: use of undeclared identifier 'ptl'
auto lambda13 = [](){return ptl>0.4 && abs(thetazl)<0.6981317007977318
                            ^
input_line_125:2:44: error: use of undeclared identifier 'thetazl'
auto lambda13 = [](){return ptl>0.4 && abs(thetazl)<0.6981317007977318
                                           ^
input_line_129:2:29: error: use of undeclared identifier 'ptl'
auto lambda13 = [](){return ptl>0.4 && abs(thetazl)<0.6981317007977318
                            ^
input_line_129:2:44: error: use of undeclared identifier 'thetazl'
auto lambda13 = [](){return ptl>0.4 && abs(thetazl)<0.6981317007977318
                                           ^
input_line_130:2:29: error: use of undeclared identifier 'ptl'
auto lambda13 = [](){return ptl>0.4 && abs(thetazl)<0.6981317007977318
                            ^
input_line_130:2:44: error: use of undeclared identifier 'thetazl'
auto lambda13 = [](){return ptl>0.4 && abs(thetazl)<0.6981317007977318
    

In [ ]:
df_bruh = df_gst_GEM21_11b_all.Filter(f"ptl>{lep_accept_pt} && abs(thetazl)<{lep_accept_angle}")
histo_bruh = df_bruh.Histo2D(("","",100,0,2.0,10,0,10), "kef[pdgf==2212]","kef[pdgf==2212].size()")
histo_bruh.Draw()

In [ ]:
#df_gst_GEM21_11b_proton = df_gst_GEM21_11b_all.Filter(f"kef[pdgf==2212] && kef>{hadron_accept_ke}")
#proton_plot = df_gst_GEM21_11b_proton.Histo2D(("","",100,0,2.0,10,0,10),"kef[pdgf==2212]","kef[pdgf==2212].size()")
#c.SetLogx()
#c.SetLogy()
#n_proton_plot.Draw("colz")
#c.Draw()


In [ ]:
#etransfer_bins = array.array('d',np.logspace(np.log10(0.05),np.log10(4),100).tolist())
#q2transfer_bins = array.array('d',np.logspace(np.log10(0.02),np.log10(10),100).tolist())
#print(len(q2transfer_bins))
#df_gst_GEM21_11b_accept = df_gst_GEM21_11b_all.Filter("ptl">0.4)
#print(len(df_gst_GEM21_11b_accept))
#h_lep_q2vw_accept = df_gst_GEM21_11b_accept.Histo2D(("","",len(etransfer_bins)-1,etransfer_bins,
                                            #len(q2transfer_bins)-1,q2transfer_bins),
                                     # "energy_transfer","Q2")
#h_lep_q2vw_accept.SetTitle("Events passing electron acceptance;Energy Transfer (GeV);Q^{2} (GeV/c)^{2}")
#h_lep_q2vw_accept.Draw("colz")
#c.SetLogx()
#c.SetLogy()
#c.Draw()

In [ ]:
#notes
#n_events_passing_cut = df_gst.Filter("my_cut_here==0").Count()
#n_entries_in_hist = my_hist.GetEntries()
test_df = df_gst_GEM21_11b_all.Define("n_proton_accept",f"kef[pdgf==2212 && kef>{hadron_accept_ke}].size()").Filter("n_proton_accept>0")